In [1]:
!pip install selenium beautifulsoup4
!pip install pandas

!pip install XlsxWriter


In [2]:
import re
import time
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import requests

# Tạo service
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)
# Mở Google
driver.get("https://google.com")

 
# Chờ cho thanh tìm kiếm xuất hiện
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "gLFyf"))
)

# Tương tác với thanh tìm kiếm
input_element = driver.find_element(By.CLASS_NAME, "gLFyf")
input_element.send_keys("nguyenkim" + Keys.ENTER)

# Xử lý CAPTCHA bằng tay (nếu có capcha)
#print("Chờ bạn xử lý CAPTCHA...")
#time.sleep(30)  # Điều chỉnh thời gian tùy ý
#print("Tiếp tục sau khi CAPTCHA đã được xử lý.")

# Chờ kết quả tìm kiếm và nhấp vào liên kết
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng"))
)
click_result = driver.find_element(By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng")
click_result.click()
# Chờ trang sản phẩm tải xong
time.sleep(10)






In [3]:
def check_internet():
    url = 'http://www.google.com'
    timeout = 5
    try:
        requests.get(url, timeout=timeout)
        return True
    except (requests.ConnectionError, requests.Timeout):
        return False

In [4]:
# Cuộn đến phần tử cần lấy dữ liệu
scroll = driver.find_element(By.CLASS_NAME,'nk-product-cate-homepage')
driver.execute_script("arguments[0].scrollIntoView();", scroll)
time.sleep(3)

# Lấy link các danh mục sản phẩm
# Tạo danh sách rỗng với tên biến là link1
link1 = []
# Biến cl_tags để tìm tất cả phần tử có class name là nl-content
cl_tags = driver.find_elements(By.CLASS_NAME, 'nk-content')
# Tạo vòng lập for với biến tag để truy xuất dữ liệu trong tất cả phần tử cl_tags
for tag in cl_tags:
    try:
        # Biến a_tags để lấy tất cả thẻ <a>
        a_tags = tag.find_elements(By.TAG_NAME, 'a')
        # Vòng lập for để lấy từng thẻ <a>
        for a_tag in a_tags:
            # Biến href được gán giá trị từ thẻ <a> với thuộc tính là href 
            href = a_tag.get_attribute('href')
            if href:
                # Thêm phần tử href vào link1
                link1.append(href)
    # Xử lý ngoại lệ            
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

In [5]:
# Kết nối đến SQLite (tạo file db mới nếu chưa tồn tại)
conn = sqlite3.connect('nguyenkim.db')
cursor = conn.cursor()

# Tạo bảng NGUYENKIM (danh mục sản phẩm và đường dẫn)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS NGUYENKIM (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        danh_muc TEXT NOT NULL,
        link TEXT NOT NULL
    )
''')

# Thêm dữ liệu vào bảng NGUYENKIM từ danh sách 'link1' (danh mục sản phẩm và đường dẫn)
for i, link in enumerate(link1):
    danh_muc = f"Danh mục {i+1}"  # Đặt tên giả cho từng danh mục
    cursor.execute('INSERT INTO NGUYENKIM (danh_muc, link) VALUES (?, ?)', (danh_muc, link))

# Lưu thay đổi
conn.commit()

In [6]:
# Tạo danh sách product_links để lưu link của từng sản phẩm cụ thể
product_links = []

# Tạo vòng lặp để duyệt qua từng link danh mục
for link in link1:  # Duyệt qua tất cả các danh mục
    # Mở link danh mục
    driver.get(link)

    # Vòng lặp while để thực hiện lấy link cụ thể của từng sản phẩm trong danh mục
    while True:
        try:
            # Chờ để trang load dữ liệu
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid')]"))
            )
            
            # Lấy các container sản phẩm
            product_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid') and contains(@class, 'nk-product-collection')]")
            
            # Lấy từng container và các thẻ <a> chứa liên kết sản phẩm
            for container in product_containers:
                try:
                    # Tìm tất cả các thẻ <a> trong container
                    a_tags = container.find_elements(By.TAG_NAME, 'a')
                    
                    # Lấy href của từng thẻ <a> và thêm vào danh sách nếu chưa có
                    for a_tag in a_tags:
                        href = a_tag.get_attribute('href')
                        if href and href not in product_links:
                            product_links.append(href)
                            print(f"Link thu được: {href}")
                
                except Exception as e:
                    print(f"Lỗi khi lấy link từ container sản phẩm: {e}")
                    continue
            
            # Tìm nút "Trang tiếp theo" để chuyển sang trang tiếp theo
            try:
                # Tìm phần tử nút "Trang tiếp theo" dựa trên lớp của nó
                next_page_button = driver.find_element(By.XPATH, "//div[contains(@class, 'NkReview_footer_col-3')]")
                
                # Cuộn trang xuống vị trí của nút "Trang tiếp theo"
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                
                # Tạm dừng để trang cuộn tới vị trí
                time.sleep(2)

                # Nhấp vào nút "Trang tiếp theo" để chuyển sang trang tiếp
                a_tag = next_page_button.find_element(By.TAG_NAME, 'a')
                href = a_tag.get_attribute('href')

                # Mở trang tiếp theo
                driver.get(href)

                # Chờ thêm để trang mới load đầy đủ
                time.sleep(3)
            
            # Nếu không còn nút "Trang tiếp theo", kết thúc vòng lặp while
            except:
                print("Không tìm thấy trang tiếp theo, kết thúc thu thập danh mục.")
                break  # Thoát khỏi vòng lặp while khi không có trang tiếp theo
        
        # Xử lý ngoại lệ nếu không thể thu thập dữ liệu
        except Exception as e:
            print(f"Lỗi thu thập: {e}")
            break  # Nếu xảy ra lỗi, thoát vòng lặp while

# In ra tổng số link sản phẩm thu thập được
print("Tổng số link sản phẩm thu thập được:", len(product_links))


Link thu được: https://www.nguyenkim.com/may-lanh-casper-inverter-tc-09is35.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=128074
Link thu được: https://www.nguyenkim.com/may-lanh-panasonic-inverter-cu-cs-pu12zkh-8m.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=127689
Link thu được: https://www.nguyenkim.com/may-lanh-reetech-1-hp-rt9-rc9-ta-bt.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=128121
Link thu được: https://www.nguyenkim.com/may-lanh-panasonic-inverter-cu-cs-xu9zkh-8.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=127668
Link thu được: https://www.nguyenkim.com/may-lanh-daikin-ftky25wmvmv.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=116296
Link thu được: https://www.nguyenkim.com/may-lan

In [7]:
print("Tổng sản phẩm: ",len(product_links))

Tổng sản phẩm:  4154


In [8]:
# Lọc link lỗi 404
filtered_links = [link for link in product_links if not link.startswith("https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=")]

In [9]:
print("Tổng sản phẩm: ",len(filtered_links))

Tổng sản phẩm:  2077


In [10]:
# Tạo danh sách để thêm thông tin chi tiết sản phẩm
data = []

# Tạo vòng lặp để lấy thông tin chi tiết sản phẩm
for op_link in filtered_links:
    while True:
        if not check_internet():
            print("Không có kết nối internet. Đang chờ...")
            while not check_internet():
                time.sleep(5)
            print("Đã khôi phục kết nối internet. Tiếp tục quá trình...")
        
        try:
            driver.get(op_link)
            time.sleep(3)

            # Lấy tên sản phẩm
            try:
                div_name = driver.find_element(By.CLASS_NAME, "wrap_name_vote")
                name = div_name.find_element(By.TAG_NAME, 'h1').text
            except:
                name = "Sản phẩm không có tên"
            
            # Lấy giá thành của sản phẩm
            try:
                div_value_final = driver.find_element(By.CLASS_NAME, "product_info_price_value-final")
                value_final = div_value_final.find_element(By.TAG_NAME, 'span').text
            except:
                value_final = ""
            
            # Lấy giá thành ban đầu của sản phẩm
            try:
                div_value_real = driver.find_element(By.CLASS_NAME, "price_promotion")
                value_real = div_value_real.find_element(By.CLASS_NAME, "product_info_price_value-real").text
            except:
                value_real = value_final

            # Lấy thông tin chi tiết
            try:
                div_specification = driver.find_element(By.CLASS_NAME, "productSpecification_table")
                html_content = div_specification.get_attribute('outerHTML')
                soup = BeautifulSoup(html_content, 'html.parser')
                rows = soup.find('tbody').find_all('tr')

                # Lấy giá trị từ các hàng chi tiết, điều chỉnh theo yêu cầu
                value_first = rows[0].find('td', class_='value').text.strip() if len(rows) > 0 else ""
                value_second = rows[1].find('td', class_='value').text.strip() if len(rows) > 1 else ""
                value_third = rows[2].find('td', class_='value').text.strip() if len(rows) > 2 else ""
                value_fourth = rows[3].find('td', class_='value').text.strip() if len(rows) > 3 else ""
                value_fifth = rows[4].find('td', class_='value').text.strip() if len(rows) > 4 else ""
                tg_bao_hanh = rows[5].find('td', class_='value').text.strip() if len(rows) > 5 else ""
                tt_chi_tiet_1 = rows[7].find('td', class_='value').text.strip() if len(rows) > 7 else ""
                tt_chi_tiet_2 = rows[8].find('td', class_='value').text.strip() if len(rows) > 8 else ""
                tt_chi_tiet_3 = rows[9].find('td', class_='value').text.strip() if len(rows) > 9 else ""
            
            except:
                value_first = value_second = value_third = value_fourth = value_fifth = ""
                tg_bao_hanh = tt_chi_tiet_1 = tt_chi_tiet_2 = tt_chi_tiet_3 = ""

            # Thêm dữ liệu vào danh sách
            data.append({
                'Tên sản phẩm': name,
                'Giá khuyến mãi': value_final,
                'Giá ban đầu': value_real,
                'Model': value_first,
                'Màu sắc': value_second,
                'Nhà sản xuất': value_third,
                'Xuất xứ': value_fourth,
                'Năm ra mắt': value_fifth,
                'TG bảo hành': tg_bao_hanh,
                'TT chi tiết 1': tt_chi_tiet_1,
                'TT chi tiết 2': tt_chi_tiet_2,
                'TT chi tiết 3': tt_chi_tiet_3
            })
        
        except Exception as e:
            print(f"Lỗi: {e}")
            break
        break

# Chuyển đổi danh sách sang DataFrame
data_df = pd.DataFrame(data)


In [11]:
# # Tạo danh sách để thêm thông tin chi tiết sản phẩm
# data = []

# # Tạo vòng lặp để lấy thông tin chi tiết sản phẩm
# for op_link in product_links:
#     try:
#         driver.get(op_link)
#         time.sleep(3)

#         # Lấy tên sản phẩm
#         try:
#             div_name = driver.find_element(By.CLASS_NAME, "wrap_name_vote")
#             name = div_name.find_element(By.TAG_NAME, 'h1').text
#         except:
#             name = "Sản phẩm không có tên"

#         # Lấy giá thành của sản phẩm
#         try:
#             div_value_final = driver.find_element(By.CLASS_NAME, "product_info_price_value-final")
#             value_final = div_value_final.find_element(By.TAG_NAME, 'span').text
#         except:
#             value_final = ""

#         # Lấy giá thành ban đầu của sản phẩm
#         try:
#             div_value_real = driver.find_element(By.CLASS_NAME, "price_promotion")
#             value_real = div_value_real.find_element(By.CLASS_NAME, "product_info_price_value-real").text
#         except:
#             value_real = value_final

#         # Lấy thông tin của sản phẩm
#         try:
#             div_specification = driver.find_element(By.CLASS_NAME, "productSpecification_table")
#             move_by_actions = ActionChains(driver)
#             move_by_actions.move_to_element(div_specification).perform()
#             time.sleep(2)

#             # Lấy toàn bộ mã HTML của phần tử chứa thông tin
#             html_content = div_specification.get_attribute('outerHTML')
#             soup = BeautifulSoup(html_content, 'html.parser')
#             t_body = soup.find('tbody')
#             rows = t_body.find_all('tr')

#             # Khởi tạo các biến giá trị
#             value_second = ""
#             value_third = ""
#             value_fourth = ""
#             value_fifth = ""
#             value_sixth = ""
#             value_eight = ""
#             value_ninth = ""
#             value_tenth = ""

#             # Duyệt qua các hàng và gán đúng cột dựa trên tiêu đề
#             for idx, row in enumerate(rows):
#                 title = row.find('td', class_='title').text.strip()
#                 value = row.find('td', class_='value').text.strip()

#                 if title == "Màu sắc:":
#                     value_second = value
#                 elif title == "Nhà sản xuất:":
#                     value_third = value
#                 elif title == "Xuất xứ:":
#                     value_fourth = value
#                 elif title == "Năm ra mắt:":
#                     value_fifth = value
#                 elif title == "TG bảo hành:":
#                     value_sixth = value
#                 elif title.startswith("TT chi tiết"):
#                     # Gán thông tin chi tiết cho từng mục
#                     if not value_eight:
#                         value_eight = value
#                     elif not value_ninth:
#                         value_ninth = value
#                     else:
#                         value_tenth = value
#         except Exception as e:
#             print(f"Lỗi khi lấy thông tin chi tiết sản phẩm: {e}")
#             # Đảm bảo tất cả các giá trị đều có giá trị mặc định nếu xảy ra lỗi
#             value_second = value_third = value_fourth = value_fifth = value_sixth = ""
#             value_eight = value_ninth = value_tenth = ""

#         # Thêm thông tin vào danh sách
#         data.append({
#             'Tên sản phẩm': name,
#             'Giá khuyến mãi': value_final,
#             'Giá ban đầu': value_real,
#             'Màu sắc': value_second,
#             'Nhà sản xuất': value_third,
#             'Xuất xứ': value_fourth,
#             'Năm ra mắt': value_fifth,
#             'TG bảo hành': value_sixth,
#             'TT chi tiết 1': value_eight,
#             'TT chi tiết 2': value_ninth,
#             'TT chi tiết 3': value_tenth
#         })

#     except Exception as e:
#         print(f"Lỗi khi truy cập vào link sản phẩm: {e}")

# # Chuyển dữ liệu sang DataFrame để kiểm tra
# data_df = pd.DataFrame(data)


In [12]:
data_df

,Tên sản phẩm,Giá khuyến mãi,Giá ban đầu,Model,Màu sắc,Nhà sản xuất,Xuất xứ,Năm ra mắt,TG bảo hành,TT chi tiết 1,TT chi tiết 2,TT chi tiết 3
0,Máy lạnh Casper Inverter 1 HP TC-09IS35,6.490.000đ,7.490.000đ,TC-09IS35,Trắng,Casper,Thái Lan,2023,36 Tháng,Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1 HP
1,Máy lạnh Panasonic Inverter 1.5 HP CU/CS-PU12Z...,13.890.000đ,15.970.000đ,CU/CS-PU12ZKH-8M,Trắng,Panasonic,Malaysia,2023,12 Tháng,Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1.5 HP
2,Máy lạnh Reetech 1 HP RT9/RC9-TA-BT,5.790.000đ,7.190.000đ,RT9-TA-BT/RC9-TA-BT,Trắng,Reetech,Thái Lan,2023,"24 tháng toàn máy, 60 tháng máy nén",Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1 HP
3,Máy lạnh Panasonic Inverter 1 HP CU/CS-XU9ZKH-8,13.990.000đ,16.090.000đ,CU/CS-XU9ZKH-8,Trắng,Panasonic,Malaysia,2023,12 Tháng,Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1 HP
4,Máy lạnh Daikin Inverter 1 HP FTKY25WMVMV,12.990.000đ,13.990.000đ,FTKY25WMVMV/RKY25WMVMV,Trắng,Daikin,Việt Nam,2022,12 Tháng,Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1 HP
...,...,...,...,...,...,...,...,...,...,...,...,...
2072,Máy duỗi tóc Dyson Corrale™ HS07 (Xám Đen/Hồng...,13.990.000đ,13.990.000đ,HS07,Xám đen/Hồng Fuchsia,Dyson,Philippines,2024,24 Tháng,Máy duỗi tóc,200W,Không
2073,Máy duỗi tóc Dyson Corrale™ (Hồng/Xám Trắng),13.990.000đ,13.990.000đ,HS07,Hồng/Xám trắng,Dyson,Philippines,2024,24 Tháng,Máy duỗi tóc,200W,Không
2074,Máy tạo kiểu tóc Dyson Airwrap Complete Long H...,14.690.000đ,15.690.000đ,HS05,Xám trắng và Đồng,Dyson,Malaysia,2022,24 Tháng,Máy tạo kiểu tóc đa năng,Đang cập nhật,Không
2075,Máy tạo kiểu tóc Dyson Airwrap Complete HS05,14.690.000đ,15.690.000đ,HS05,Dyson,Malaysia,2022,24 Tháng,Nguyễn Kim,1300W,Có,Có


In [1]:
data_df = data_df[data_df['Tên sản phẩm'] != 'Sản phẩm không có tên']
# Giả sử DataFrame của bạn là df
data_df = data_df[~data_df.apply(lambda row: row.astype(str).str.contains("Nguyễn Kim").any(), axis=1)]


NameError: name 'data_df' is not defined

In [14]:
data_df

,Tên sản phẩm,Giá khuyến mãi,Giá ban đầu,Model,Màu sắc,Nhà sản xuất,Xuất xứ,Năm ra mắt,TG bảo hành,TT chi tiết 1,TT chi tiết 2,TT chi tiết 3
0,Máy lạnh Casper Inverter 1 HP TC-09IS35,6.490.000đ,7.490.000đ,TC-09IS35,Trắng,Casper,Thái Lan,2023,36 Tháng,Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1 HP
1,Máy lạnh Panasonic Inverter 1.5 HP CU/CS-PU12Z...,13.890.000đ,15.970.000đ,CU/CS-PU12ZKH-8M,Trắng,Panasonic,Malaysia,2023,12 Tháng,Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1.5 HP
2,Máy lạnh Reetech 1 HP RT9/RC9-TA-BT,5.790.000đ,7.190.000đ,RT9-TA-BT/RC9-TA-BT,Trắng,Reetech,Thái Lan,2023,"24 tháng toàn máy, 60 tháng máy nén",Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1 HP
3,Máy lạnh Panasonic Inverter 1 HP CU/CS-XU9ZKH-8,13.990.000đ,16.090.000đ,CU/CS-XU9ZKH-8,Trắng,Panasonic,Malaysia,2023,12 Tháng,Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1 HP
4,Máy lạnh Daikin Inverter 1 HP FTKY25WMVMV,12.990.000đ,13.990.000đ,FTKY25WMVMV/RKY25WMVMV,Trắng,Daikin,Việt Nam,2022,12 Tháng,Máy lạnh 1 chiều (chỉ làm lạnh),Máy lạnh treo tường,1 HP
...,...,...,...,...,...,...,...,...,...,...,...,...
2072,Máy duỗi tóc Dyson Corrale™ HS07 (Xám Đen/Hồng...,13.990.000đ,13.990.000đ,HS07,Xám đen/Hồng Fuchsia,Dyson,Philippines,2024,24 Tháng,Máy duỗi tóc,200W,Không
2073,Máy duỗi tóc Dyson Corrale™ (Hồng/Xám Trắng),13.990.000đ,13.990.000đ,HS07,Hồng/Xám trắng,Dyson,Philippines,2024,24 Tháng,Máy duỗi tóc,200W,Không
2074,Máy tạo kiểu tóc Dyson Airwrap Complete Long H...,14.690.000đ,15.690.000đ,HS05,Xám trắng và Đồng,Dyson,Malaysia,2022,24 Tháng,Máy tạo kiểu tóc đa năng,Đang cập nhật,Không
2075,Máy tạo kiểu tóc Dyson Airwrap Complete HS05,14.690.000đ,15.690.000đ,HS05,Dyson,Malaysia,2022,24 Tháng,Nguyễn Kim,1300W,Có,Có


In [18]:


# Hàm chuẩn hóa tên bảng để đảm bảo không có ký tự không hợp lệ
def normalize_table_name(name):
    # Xóa các ký tự không phải chữ cái, chữ số và dấu gạch dưới
    return re.sub(r'\W+', '_', name).lower()

# Hàm để tạo bảng thông tin sản phẩm cho từng danh mục
def create_product_table(danh_muc_name):
    try:
        table_name = normalize_table_name(danh_muc_name)  # Chuẩn hóa tên bảng
        cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS {table_name} (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ten_san_pham TEXT,
                gia_khuyen_mai TEXT,
                gia_ban_dau TEXT,
                model TEXT,
                mau_sac TEXT,
                nha_san_xuat TEXT,
                xuat_xu TEXT,
                nam_ra_mat TEXT,
                tg_bao_hanh TEXT,
                thong_tin_chi_tiet_1 TEXT,
                thong_tin_chi_tiet_2 TEXT,
                thong_tin_chi_tiet_3 TEXT
            )
        ''')
        conn.commit()
        print(f"Table '{table_name}' created successfully.")
    except Exception as e:
        print(f"Error creating table '{table_name}': {e}")

# Hàm để thêm dữ liệu vào bảng
def insert_product_data(danh_muc_name, product_data):
    try:
        table_name = normalize_table_name(danh_muc_name)
        cursor.execute(f'''
            INSERT INTO {table_name} 
            (ten_san_pham, gia_khuyen_mai, gia_ban_dau, model, mau_sac, nha_san_xuat, xuat_xu, nam_ra_mat, tg_bao_hanh, thong_tin_chi_tiet_1, thong_tin_chi_tiet_2, thong_tin_chi_tiet_3)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            product_data['Tên sản phẩm'],
            product_data['Giá khuyến mãi'],
            product_data['Giá ban đầu'],
            product_data['Model'],
            product_data['Màu sắc'],
            product_data['Nhà sản xuất'],
            product_data['Xuất xứ'],
            product_data['Năm ra mắt'],
            product_data['TG bảo hành'],
            product_data['TT chi tiết 1'],
            product_data['TT chi tiết 2'],
            product_data['TT chi tiết 3']
        ))
        conn.commit()
        print(f"Data inserted successfully into '{table_name}'.")
    except Exception as e:
        print(f"Error inserting data into '{table_name}': {e}")

# Lặp qua từng danh mục và tạo bảng cho mỗi danh mục sản phẩm
for i, (danh_muc_name, danh_muc_data) in enumerate(zip(link1, data)):
    # Tạo bảng cho danh mục này
    create_product_table(danh_muc_name)
    
    # Lặp qua dữ liệu của từng sản phẩm trong danh mục và thêm vào bảng tương ứng
    for product in danh_muc_data:
        insert_product_data(danh_muc_name, product)


Table 'https_www_nguyenkim_com_may_lanh_' created successfully.
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices must be integers
Error inserting data into 'https_www_nguyenkim_com_may_lanh_': string indices mus

In [19]:
# Đóng kết nối khi hoàn tất
conn.close()
driver.quit()


In [20]:
data_df.to_excel("output.xlsx", index=False)

print("Dữ liệu đã được lưu vào SQLite và file Excel 'output.xlsx'")

Dữ liệu đã được lưu vào SQLite và file Excel 'output.xlsx'
